In [1]:
import pandas as pd
from pathlib import Path

In [2]:
ddf = pd.read_csv('../data/curated/historical_data.csv')
ddf = ddf.rename(columns={'Sub': 'suburb'})

popudf = pd.read_csv('../data/curated/subpopu.csv')
popudf = popudf[['POSTCODE', '2016', '2017', '2018', '2019',
                 '2020', '2021', '2022', '2023', '2024', '2025']]

incomedf = pd.read_csv('../data/curated/subincome.csv')
incomedf = incomedf[['POSTCODE', '2016', '2017', '2018', '2019',
                     '2020', '2021', '2022', '2023', '2024', '2025']]

In [3]:
# convert year to a feature column
years = ['2016', '2017', '2018', '2019',
         '2020', '2021', '2022', '2023', '2024', '2025']

new_popudf = pd.DataFrame(columns=['POSTCODE', 'Year', 'population'])
for year in years:
    df = popudf[['POSTCODE', year]]
    df = df.rename(columns={year: 'population'})
    df['Year'] = year
    new_popudf = pd.concat([new_popudf, df], axis=0)

In [4]:
# convert year to a feature column
new_incomedf = pd.DataFrame(columns=['POSTCODE', 'Year', 'income'])
for year in years:
    df = incomedf[['POSTCODE', year]]
    df = df.rename(columns={year: 'income'})
    df['Year'] = year
    new_incomedf = pd.concat([new_incomedf, df], axis=0)

In [5]:
df_merged = pd.merge(new_incomedf, new_popudf, on=['POSTCODE', 'Year'], how='inner')

In [6]:
sub_postcodeinfo = pd.read_csv('../data/raw/realestate.csv')
sub_postcodeinfo = sub_postcodeinfo[['suburb', 'postcode']]
sub_postcode = sub_postcodeinfo.groupby(['suburb', 'postcode']).count().reset_index()
sub_postcode = sub_postcode.rename(columns={'postcode': 'POSTCODE'})

In [7]:
# convert postcode to suburb
df_merged = pd.merge(df_merged, sub_postcode, on='POSTCODE', how='inner')
df_merged = df_merged[['suburb', 'Year', 'population', 'income']]
df_merged = df_merged.groupby(['suburb', 'Year']).agg({'population': 'sum',
                                                       'income': 'mean'}).reset_index()

In [8]:
df_merged['Year'] = df_merged['Year'].astype(int)
df_model = df_merged[df_merged['Year'] <= 2020]
df_predict = df_merged[df_merged['Year'] > 2020]

In [9]:
subdf = ddf.groupby('suburb').count().reset_index()
subdf = subdf['suburb']

In [10]:

df_predict = pd.merge(df_predict, subdf, on=['suburb'], how='inner')

In [11]:
typelist = ddf.groupby('Type').count().reset_index()
typelist = typelist['Type'].tolist()

In [12]:
newdf_all = pd.DataFrame(columns=['suburb', 'Year', 'population', 'income', 'Type'])
for type in typelist:
    newdf = df_predict.copy()
    newdf['Type'] = type
    newdf_all = pd.concat([newdf_all, newdf], axis=0)

In [13]:
filepath = Path('../data/curated/ques2predictdata.csv')
newdf_all.to_csv(filepath, index=False)

In [14]:
df_model = pd.merge(df_model, ddf, on=['Year', 'suburb'], how='inner')
filepath = Path('../data/curated/ques2data.csv')
df_model.to_csv(filepath, index=False)